# Option 2: Test DenseNet Submission## GoalSubmit DenseNet (Val R²=0.6605) to Kaggle to test the validation-test gap theory.## HypothesisThe validation-test gap (-0.175) affects all models similarly.**Expected**: DenseNet Kaggle R² ~ 0.49 (worse than baseline 0.51)This would confirm that even "good" validation scores don't guarantee good Kaggle scores.---

## Setup

In [ ]:
import pandas as pdimport numpy as npimport torchimport torch.nn as nnfrom torch.utils.data import Dataset, DataLoaderimport torchvision.transforms as transformsimport torchvision.models as modelsfrom PIL import Imagefrom tqdm.auto import tqdmfrom datetime import datetimedevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f"Device: {device}")TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']BATCH_SIZE = 16

## Load Data & Define Model

In [ ]:
# Load test datatest_df = pd.read_csv('../../competition/test.csv')test_df['full_image_path'] = test_df['image_path'].apply(lambda x: f'../../competition/{x}')unique_images = test_df[['image_path', 'full_image_path']].drop_duplicates()print(f"Test images: {len(unique_images)}")# Test datasetclass TestDataset(Dataset):    def __init__(self, image_paths):        self.image_paths = image_paths        self.transform = transforms.Compose([            transforms.Resize((224, 224)),            transforms.ToTensor(),            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])        ])    def __len__(self):        return len(self.image_paths)    def __getitem__(self, idx):        img = Image.open(self.image_paths[idx]).convert('RGB')        return self.transform(img)test_dataset = TestDataset(unique_images['full_image_path'].tolist())test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)print(f"✓ Test dataset: {len(test_dataset)} images")

In [ ]:
# Define model (must match training architecture)
class AuxiliaryPretrainedModel(nn.Module):
    def __init__(self, num_outputs=5, hidden_dim=256, dropout=0.2, num_states=4, num_species=15):
        super().__init__()
        # DenseNet121 backbone
        densenet = models.densenet121(pretrained=True)
        self.backbone = nn.Sequential(
            densenet.features, nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1))
        )
        feature_dim = 1024
        
        # Auxiliary heads (required for loading checkpoint, even though we won't use them)
        self.ndvi_head = nn.Linear(feature_dim, 1)
        self.height_head = nn.Linear(feature_dim, 1)
        self.weather_head = nn.Linear(feature_dim, 14)
        self.state_head = nn.Linear(feature_dim, num_states)
        self.species_head = nn.Linear(feature_dim, num_species)
        
        # Biomass head (this is what we actually use)
        self.biomass_head = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_outputs)
        )
    
    def forward(self, x, mode='biomass'):
        features = self.backbone(x).flatten(1)
        if mode == 'auxiliary':
            return {
                'ndvi': self.ndvi_head(features),
                'height': self.height_head(features),
                'weather': self.weather_head(features),
                'state': self.state_head(features),
                'species': self.species_head(features)
            }
        else:
            return self.biomass_head(features)

model = AuxiliaryPretrainedModel()
print("✓ Model defined (with auxiliary heads for checkpoint compatibility)")

## Load Checkpoint & Generate Predictions

In [ ]:
# Load trained checkpointcheckpoint_path = '../../model4b_4b8_DenseNet_phase2_best.pth'model.load_state_dict(torch.load(checkpoint_path, map_location=device))model = model.to(device)model.eval()print(f"✓ Loaded: {checkpoint_path}")# Normalization stats (from training)target_means = torch.tensor([27.49, 12.01, 6.26, 33.74, 45.75])target_stds = torch.tensor([26.19, 12.50, 11.75, 25.62, 28.86])print("\nGenerating predictions...")all_preds = []with torch.no_grad():    for images in tqdm(test_loader):        images = images.to(device)        outputs = model(images)        outputs_denorm = outputs.cpu() * target_stds + target_means        outputs_denorm = torch.clamp(outputs_denorm, min=0)        all_preds.append(outputs_denorm.numpy())predictions = np.vstack(all_preds)print(f"✓ Predictions: {predictions.shape}")

## Create Submission

In [ ]:
# Create submission in long formatsubmission_rows = []for idx, img_path in enumerate(unique_images['image_path'].tolist()):    image_id = img_path.split('/')[-1].replace('.jpg', '')    for target_idx, target_name in enumerate(TARGET_COLS):        submission_rows.append({            'sample_id': f"{image_id}__{target_name}",            'target': predictions[idx, target_idx]        })submission = pd.DataFrame(submission_rows)timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')filename = f'../../submission_densenet_{timestamp}.csv'submission.to_csv(filename, index=False)print(f"\n✅ Submission saved: {filename}")print(f"   Rows: {len(submission)}")print(f"   Expected Kaggle R²: ~0.49 (validation was 0.6605)")print(f"\n📊 If Kaggle score < 0.51 → Confirms validation-test gap theory")